In [195]:
import pandas as pd
import matplotlib.pylab as plt
import string                            #导入字符串模块

from itertools import chain

# 数据预处理

In [196]:
tab1 = "./hair_dryer.tsv"

df = pd.read_csv(tab1, sep='\t', header=0)

## 删除无效信息

In [197]:
df = df[~(df['vine'].str.contains("N") & df['verified_purchase'].str.contains("N"))]

In [198]:
df = df[(df['total_votes'] > 0) & (df['helpful_votes'] > 0)]

In [199]:
df = df.groupby('product_parent').filter(lambda x: len(x) > 1)

In [200]:
print(len(df))

2775


# 整体文本分析

In [201]:
pattern = r"\&\#[0-9]+\;"

df["preprocessed"] = df["review_body"].str.replace(pat=pattern, repl="", regex=True)

In [202]:
import re
import nltk

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet

#import nltk resources
resources = ["wordnet", "stopwords", "punkt", \
             "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"]

for resource in resources:
    try:
        nltk.data.find("tokenizers/" + resource)
    except LookupError:
        nltk.download(resource)

#create Lemmatizer object
lemma = WordNetLemmatizer()

def lemmatize_word(tagged_token):
    """ Returns lemmatized word given its tag"""
    root = []
    for token in tagged_token:
        tag = token[1][0]
        word = token[0]
        if tag.startswith('J'):
            root.append(lemma.lemmatize(word, wordnet.ADJ))
        elif tag.startswith('V'):
            root.append(lemma.lemmatize(word, wordnet.VERB))
        elif tag.startswith('N'):
            root.append(lemma.lemmatize(word, wordnet.NOUN))
        elif tag.startswith('R'):
            root.append(lemma.lemmatize(word, wordnet.ADV))
        else:          
            root.append(word)
    return root




[nltk_data] Downloading package wordnet to /home/alphonse/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/alphonse/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


In [203]:
def lemmatize_doc(document):
    """ Tags words then returns sentence with lemmatized words"""
    lemmatized_list = []
    tokenized_sent = sent_tokenize(document)
    for sentence in tokenized_sent:
        no_punctuation = re.sub(r"[`'\",.!?()]", " ", sentence)
        tokenized_word = word_tokenize(no_punctuation)
        tagged_token = pos_tag(tokenized_word)
        lemmatized = lemmatize_word(tagged_token)
        lemmatized_list.extend(lemmatized)
    return " ".join(lemmatized_list)

In [204]:
#apply our functions
df["preprocessed"] = df["preprocessed"].apply(lambda row: lemmatize_doc(row))

print(df["preprocessed"].iloc[45])

Found this gem in a hotel in Savannah and like it so much I take a photo and find it on Amazon Bought two one for each bathroom


In [205]:
from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

df["preprocessed"] = df["preprocessed"].apply(remove_accent)

print(df["preprocessed"].iloc[465])

Quickly dry your hair without fry it~


In [206]:
pattern = r"[^\w\s]"

df["preprocessed"] = df["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

print(df["preprocessed"].iloc[465])

Quickly dry your hair without fry it 


In [207]:
df["preprocessed"] = df["preprocessed"].str.lower()

print(df["preprocessed"].iloc[465])

quickly dry your hair without fry it 


In [208]:
# from nltk.corpus import stopwords

# stop_words = stopwords.words("english")

# stop_words = [word.replace("\'", "") for word in stop_words]

# print(f"sample stop words: {stop_words[:15]} \n")

# remove_stop_words = lambda row: " ".join([token for token in row.split(" ") \
#                                           if token not in stop_words])
# df["preprocessed"] = df["preprocessed"].apply(remove_stop_words)

# print(df["preprocessed"].iloc[465])

In [209]:
pattern = r"[\s]+"

df["preprocessed"] = df["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

print(df["preprocessed"].iloc[465])

quickly dry your hair without fry it 


In [210]:
corpora = df["preprocessed"].values
tokenized = [corpus.split(" ") for corpus in corpora]

for review in tokenized:
    while '' in review:
        review.remove('')

# text = list(chain(*tokenized))

text = []
for i in range(0, int(len(df) / 20)):
    text = text + tokenized[i]


print(text)


['i', 'find', 'everything', 'go', 'well', 'except', 'the', 'plug', 'why', 'the', 'left', 'and', 'right', 'be', 'opposite', 'and', 'i', 'have', 'to', 'put', 'the', 'plug', 'upside', 'down', 'for', 'charge', 'another', 'flaw', 'be', 'the', 'big', 'noise', 'reckon', 'i', 'have', 'have', 'this', 'about', 'a', 'year', 'now', 'at', 'least', '7', 'month', 'works', 'great', 'i', 'use', 'it', '5', 'day', 'a', 'week', 'blow', 'hot', 'air', 'doesnt', 'overheat', 'isnt', 'to', 'big', 'come', 'quick', 'didnt', 'cost', 'much', 'get', 'you', 'one', 'you', 'will', 'like', 'it', 'great', 'product', 'took', 'me', 'anywhere', 'between', '30', '40', 'min', 'to', 'blow', 'dry', 'my', 'client', 'hair', 'with', 'my', 'old', 'blow', 'drier', 'with', 'this', 'new', 'one', 'its', 'cut', 'the', 'time', 'in', 'half', 'it', 's', 'have', 'lot', 'of', 'power', 'great', 'deal', 'n', 'it', 's', 'super', 'cute', 'update', '7', '8', 'month', 'later', 'it', 'only', 'blow', 'cold', 'air', 'i', 'wouldn', 't', 'recommend', 

### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

In [211]:
POS_tag = nltk.pos_tag(text)

print ("Tokenized Text with POS tags: \n")
print (POS_tag)

Tokenized Text with POS tags: 

[('i', 'JJ'), ('find', 'VBP'), ('everything', 'NN'), ('go', 'VBP'), ('well', 'RB'), ('except', 'IN'), ('the', 'DT'), ('plug', 'NN'), ('why', 'WRB'), ('the', 'DT'), ('left', 'NN'), ('and', 'CC'), ('right', 'JJ'), ('be', 'VB'), ('opposite', 'JJ'), ('and', 'CC'), ('i', 'NNS'), ('have', 'VBP'), ('to', 'TO'), ('put', 'VB'), ('the', 'DT'), ('plug', 'NN'), ('upside', 'RB'), ('down', 'RB'), ('for', 'IN'), ('charge', 'NN'), ('another', 'DT'), ('flaw', 'NN'), ('be', 'VB'), ('the', 'DT'), ('big', 'JJ'), ('noise', 'NN'), ('reckon', 'NN'), ('i', 'NN'), ('have', 'VBP'), ('have', 'VBN'), ('this', 'DT'), ('about', 'IN'), ('a', 'DT'), ('year', 'NN'), ('now', 'RB'), ('at', 'IN'), ('least', 'JJS'), ('7', 'CD'), ('month', 'NN'), ('works', 'VBZ'), ('great', 'JJ'), ('i', 'NN'), ('use', 'VBP'), ('it', 'PRP'), ('5', 'CD'), ('day', 'NN'), ('a', 'DT'), ('week', 'NN'), ('blow', 'NN'), ('hot', 'JJ'), ('air', 'NN'), ('doesnt', 'VBZ'), ('overheat', 'NN'), ('isnt', 'NN'), ('to', 'TO')

### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

In [212]:
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
print (lemmatized_text)

Text tokens after lemmatization of adjectives and nouns: 

['i', 'find', 'everything', 'go', 'well', 'except', 'the', 'plug', 'why', 'the', 'left', 'and', 'right', 'be', 'opposite', 'and', 'i', 'have', 'to', 'put', 'the', 'plug', 'upside', 'down', 'for', 'charge', 'another', 'flaw', 'be', 'the', 'big', 'noise', 'reckon', 'i', 'have', 'have', 'this', 'about', 'a', 'year', 'now', 'at', 'least', '7', 'month', 'work', 'great', 'i', 'use', 'it', '5', 'day', 'a', 'week', 'blow', 'hot', 'air', 'doesnt', 'overheat', 'isnt', 'to', 'big', 'come', 'quick', 'didnt', 'cost', 'much', 'get', 'you', 'one', 'you', 'will', 'like', 'it', 'great', 'product', 'took', 'me', 'anywhere', 'between', '30', '40', 'min', 'to', 'blow', 'dry', 'my', 'client', 'hair', 'with', 'my', 'old', 'blow', 'drier', 'with', 'this', 'new', 'one', 'it', 'cut', 'the', 'time', 'in', 'half', 'it', 's', 'have', 'lot', 'of', 'power', 'great', 'deal', 'n', 'it', 's', 'super', 'cute', 'update', '7', '8', 'month', 'later', 'it', 'only',

### POS tagging for Filtering

The <b>lemmatized text</b> is <b>POS tagged</b> here. The tags will be used for filtering later on.

In [213]:
POS_tag = nltk.pos_tag(lemmatized_text)

print ("Lemmatized text with POS tags: \n")
print (POS_tag)

Lemmatized text with POS tags: 

[('i', 'JJ'), ('find', 'VBP'), ('everything', 'NN'), ('go', 'VBP'), ('well', 'RB'), ('except', 'IN'), ('the', 'DT'), ('plug', 'NN'), ('why', 'WRB'), ('the', 'DT'), ('left', 'NN'), ('and', 'CC'), ('right', 'JJ'), ('be', 'VB'), ('opposite', 'JJ'), ('and', 'CC'), ('i', 'NNS'), ('have', 'VBP'), ('to', 'TO'), ('put', 'VB'), ('the', 'DT'), ('plug', 'NN'), ('upside', 'RB'), ('down', 'RB'), ('for', 'IN'), ('charge', 'NN'), ('another', 'DT'), ('flaw', 'NN'), ('be', 'VB'), ('the', 'DT'), ('big', 'JJ'), ('noise', 'NN'), ('reckon', 'NN'), ('i', 'NN'), ('have', 'VBP'), ('have', 'VBN'), ('this', 'DT'), ('about', 'IN'), ('a', 'DT'), ('year', 'NN'), ('now', 'RB'), ('at', 'IN'), ('least', 'JJS'), ('7', 'CD'), ('month', 'NN'), ('work', 'NN'), ('great', 'JJ'), ('i', 'NN'), ('use', 'VBP'), ('it', 'PRP'), ('5', 'CD'), ('day', 'NN'), ('a', 'DT'), ('week', 'NN'), ('blow', 'NN'), ('hot', 'JJ'), ('air', 'NN'), ('doesnt', 'VBZ'), ('overheat', 'NN'), ('isnt', 'NN'), ('to', 'TO'),

## POS Based Filtering

Any word from the lemmatized text, which isn't a noun, adjective, or gerund (or a 'foreign word'), is here
considered as a <b>stopword</b> (non-content). This is based on the assumption that usually keywords are noun,
adjectives or gerunds. 

Punctuations are added to the stopword list too.

In [214]:
stopwords = []

# wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 
wanted_POS = ['NN', 'NNS', 'NNP', 'NNPS']
for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

### Complete stopword generation

Even if we remove the aforementioned stopwords, still some extremely common nouns, adjectives or gerunds may
remain which are very bad candidates for being keywords (or part of it). 

An external file constituting a long list of stopwords is loaded and all the words are added with the previous
stopwords to create the final list 'stopwords-plus' which is then converted into a set. 

(Source of stopwords data: https://www.ranks.nl/stopwords)

Stopwords-plus constitute the sum total of all stopwords and potential phrase-delimiters. 

(The contents of this set will be later used to partition the lemmatized text into n-gram phrases. But, for now, I will simply remove the stopwords, and work with a 'bag-of-words' approach. I will be developing the graph using unigram texts as vertices)

In [215]:
stopword_file = open("long_stopwords.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

### Removing Stopwords 

Removing stopwords from lemmatized_text. 
Processeced_text condtains the result.

In [216]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print (processed_text)

['plug', 'left', 'plug', 'flaw', 'noise', 'reckon', 'year', 'month', 'day', 'week', 'air', 'isnt', 'didnt', 'cost', 'product', 'min', 'client', 'drier', 'time', 'lot', 'power', 'deal', 'month', 'air', 'drier', 'price', 'conair', 'makeover', 'instance', 'replica', 'temperature', 'adjustment', 'fan', 'order', 'air', 'button', 'person', 'finger', 'button', 'sliding', 'conair', 'bathroom', 'watt', 'power', 'air', 'controller', 'investment', 'time', 'dust', 'year', 'suitcase', 'style', 'comb', 'frill', 'drier', 'time', 'color', 'winter', 'blaze', 'glory', 'refuse', 'selection', 'product', 'appendage', 'airplane', 'wind', 'tunnel', 'bother', 'drawer', 'space', 'wind', 'tunnel', 'pretend', 'wind', 'tunnel', 'miniature', 'airplane', 'thing', 'blast', 'reason', 'minute', 'wind', 'tunnel', 'style', 'space', 'product', 'bio', 'problem', 'bio', 'star', 'hairdryer', 'price', 'delivery', 'packaging', 'email', 'power', 'blast', 'deal', 'product', 'hairdryer', 'timing', 'ghd', 'downside', 'temperature

## Vocabulary Creation
Vocabulary will only contain unique words from processed_text.

In [217]:
vocabulary = list(set(processed_text))
print (vocabulary)

['problem', 'dark', 'complaint', 'year', 'dey', 'fan', 'rusk', 'lint', 'energy', 'sliding', 'w8less', 'number', 'minipro', 'basis', 'blast', 'fuss', 'comb', 'sign', 'space', 'clunky', 'regard', 'glory', 'pain', 'device', 'conair', 'mode', 'treat', 'gem', 'button', 'pole', 'opening', 'job', 'gimmick', 'miniature', 'dyer', 'time', 'tame', 'bleach', 'humidity', 'steamer', 'mus', 'choice', 'damage', 'hour', 'twinturbo', 'cable', 'lot', 'direction', 'bathroom', 'success', 'ac9096', 'pixie', 'drain', 'oster', 'hurry', 'bottom', 'root', 'bag', 'selling', 'risk', 'shriek', 'heist', 'hurricane', 'tool', 'brand', 'plunge', 'life', 'thumb', 'ion', 'price', 'people', 'confidence', 'wind', 'straggler', 'flame', 'pick', 'melt', 'con', 'leg', 'beat', 'feedback', 'aid', 'drawer', 'shiney', 'outlet', 'potential', 'glowing', 'crap', 'ti', 'motor', 'routine', 'technology', 'company', 'stint', 'width', 'scalp', 'age', 'occasion', 'salon', 'week', 'burner', 'case', 'voltage', 'forget', 'teeth', 'attempt', 

### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [218]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


### Calculating weighted summation of connections of a vertex

inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.

In [219]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

### Scoring Vertices

The formula used for scoring a vertex represented by i is:

score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i. 

d is the damping factor.

The score is iteratively updated until convergence. 

In [220]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break

Converging at iteration 36....


In [221]:
for i in range(0,vocab_len):
    print("Score of "+vocabulary[i]+": "+str(score[i]))

Score of problem: 1.7436581
Score of dark: 0.5233462
Score of complaint: 0.4682648
Score of year: 8.116927
Score of dey: 0.45476112
Score of fan: 1.7101119
Score of rusk: 1.5332575
Score of lint: 1.2095855
Score of energy: 0.43603814
Score of sliding: 0.48386353
Score of w8less: 0.88189733
Score of number: 0.82646847
Score of minipro: 0.52555555
Score of basis: 0.9887321
Score of blast: 1.0390311
Score of fuss: 0.5774302
Score of comb: 0.9501513
Score of sign: 0.5024107
Score of space: 1.5295525
Score of clunky: 0.5099423
Score of regard: 0.5097583
Score of glory: 0.64465505
Score of pain: 0.56077665
Score of device: 0.4534389
Score of conair: 1.8613368
Score of mode: 0.64805704
Score of treat: 0.47659415
Score of gem: 0.5258796
Score of button: 3.5877612
Score of pole: 0.5487398
Score of opening: 0.4620421
Score of job: 2.3157744
Score of gimmick: 0.57945466
Score of miniature: 0.48660254
Score of dyer: 0.8333449
Score of time: 12.914936
Score of tame: 0.5148198
Score of bleach: 0.441

### Phrase Partiotioning

Paritioning lemmatized_text into phrases using the stopwords in it as delimeters.
The phrases are also candidates for keyphrases to be extracted. 

In [222]:
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print("Partitioned Phrases (Candidate Keyphrases): \n")
print(phrases)

Partitioned Phrases (Candidate Keyphrases): 

[['plug'], ['left'], ['plug'], ['flaw'], ['noise', 'reckon'], ['year'], ['month'], ['day'], ['week'], ['air'], ['isnt'], ['didnt', 'cost'], ['product'], ['min'], ['client'], ['drier'], ['time'], ['lot'], ['power'], ['deal'], ['month'], ['air'], ['drier'], ['price'], ['conair'], ['makeover'], ['instance'], ['replica'], ['temperature', 'adjustment'], ['fan'], ['order'], ['air'], ['button'], ['person'], ['finger'], ['button'], ['sliding'], ['conair'], ['bathroom'], ['watt'], ['power'], ['air', 'controller'], ['investment'], ['time'], ['dust'], ['year'], ['suitcase'], ['style'], ['comb'], ['frill'], ['drier'], ['time'], ['color'], ['winter'], ['blaze'], ['glory'], ['refuse'], ['selection'], ['product'], ['appendage'], ['airplane', 'wind', 'tunnel'], ['bother'], ['drawer', 'space'], ['wind', 'tunnel'], ['pretend'], ['wind', 'tunnel'], ['miniature', 'airplane'], ['thing'], ['blast'], ['reason'], ['minute'], ['wind', 'tunnel'], ['style'], ['space'

### Create a list of unique phrases.
Repeating phrases\keyphrase candidates has no purpose here, anymore.

In [223]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print("Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)

Unique Phrases (Candidate Keyphrases): 

[['plug'], ['left'], ['flaw'], ['noise', 'reckon'], ['year'], ['month'], ['day'], ['week'], ['air'], ['isnt'], ['didnt', 'cost'], ['product'], ['min'], ['client'], ['drier'], ['time'], ['lot'], ['power'], ['deal'], ['price'], ['conair'], ['makeover'], ['instance'], ['replica'], ['temperature', 'adjustment'], ['fan'], ['order'], ['button'], ['person'], ['finger'], ['sliding'], ['bathroom'], ['watt'], ['air', 'controller'], ['investment'], ['dust'], ['suitcase'], ['style'], ['comb'], ['frill'], ['color'], ['winter'], ['blaze'], ['glory'], ['refuse'], ['selection'], ['appendage'], ['airplane', 'wind', 'tunnel'], ['bother'], ['drawer', 'space'], ['wind', 'tunnel'], ['pretend'], ['miniature', 'airplane'], ['thing'], ['blast'], ['reason'], ['minute'], ['space'], ['bio'], ['problem'], ['star'], ['hairdryer'], ['price', 'delivery'], ['packaging'], ['email'], ['timing'], ['ghd'], ['downside'], ['temperature'], ['drawback'], ['placement'], ['locate'], ['b

### Thinning the list of candidate-keyphrases.
Removing single word keyphrases-candidates that are present multi-word alternatives.

In [224]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print("Thinned Unique Phrases (Candidate Keyphrases): \n")
print(unique_phrases)  

Thinned Unique Phrases (Candidate Keyphrases): 

[['left'], ['flaw'], ['noise', 'reckon'], ['day'], ['isnt'], ['didnt', 'cost'], ['product'], ['min'], ['client'], ['drier'], ['lot'], ['power'], ['deal'], ['conair'], ['makeover'], ['instance'], ['replica'], ['temperature', 'adjustment'], ['order'], ['person'], ['finger'], ['sliding'], ['bathroom'], ['watt'], ['air', 'controller'], ['investment'], ['dust'], ['suitcase'], ['comb'], ['frill'], ['winter'], ['blaze'], ['glory'], ['refuse'], ['selection'], ['appendage'], ['airplane', 'wind', 'tunnel'], ['bother'], ['drawer', 'space'], ['wind', 'tunnel'], ['pretend'], ['miniature', 'airplane'], ['blast'], ['reason'], ['bio'], ['problem'], ['hairdryer'], ['price', 'delivery'], ['packaging'], ['email'], ['timing'], ['ghd'], ['downside'], ['drawback'], ['placement'], ['locate'], ['bottom'], ['thumb'], ['temperature', 'button'], ['reach'], ['hole'], ['side'], ['tool', 'bonnet'], ['hose'], ['notch'], ['ring'], ['heist'], ['knob'], ['pole'], ['wheel

### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [225]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: 'left', Score: 0.4123534560203552
Keyword: 'flaw', Score: 0.8771644234657288
Keyword: 'noise reckon', Score: 3.082283526659012
Keyword: 'day', Score: 3.5466365814208984
Keyword: 'isnt', Score: 0.4600785970687866
Keyword: 'didnt cost', Score: 2.4737287759780884
Keyword: 'product', Score: 10.27767562866211
Keyword: 'min', Score: 0.8280441164970398
Keyword: 'client', Score: 0.4668336808681488
Keyword: 'drier', Score: 2.2306318283081055
Keyword: 'lot', Score: 2.9045815467834473
Keyword: 'power', Score: 5.939521312713623
Keyword: 'deal', Score: 2.195890426635742
Keyword: 'conair', Score: 1.8613368272781372
Keyword: 'makeover', Score: 0.5514674186706543
Keyword: 'instance', Score: 0.5747207403182983
Keyword: 'replica', Score: 0.5674861669540405
Keyword: 'temperature adjustment', Score: 2.4336069524288177
Keyword: 'order', Score: 1.176138997077942
Keyword: 'person', Score: 1.6464309692382812
Keyword: 'finger', Score: 0.49023377895355225
Keyword: 'sliding', Score: 0.4838635325431824
K

### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [229]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 100

print("Keywords:\n")

for i in range(0,keywords_num):
    print(str(keywords[sorted_index[i]])+", Score: "+str(phrase_scores[sorted_index[i]]) )

Keywords:

week time, Score: 16.55882477760315
dey time, Score: 13.36969718337059
product, Score: 10.27767562866211
year warranty, Score: 10.018328070640564
con air, Score: 8.45044469833374
month enever, Score: 7.5322277545928955
air flow, Score: 7.010872960090637
air jet, Score: 6.909917891025543
star customer service, Score: 6.753957629203796
air controller, Score: 6.640643626451492
air throughput, Score: 6.606684297323227
price delivery, Score: 6.178928375244141
power, Score: 5.939521312713623
price needle, Score: 5.885395884513855
temperature button, Score: 5.525351166725159
damage control, Score: 5.409917116165161
hand piece, Score: 5.37585723400116
quality control, Score: 5.361099481582642
shot button, Score: 5.1755348443984985
unit, Score: 5.051833152770996
tool bonnet, Score: 4.556514024734497
protection technology, Score: 4.446175456047058
airplane wind tunnel, Score: 4.3624749183654785
quality tool, Score: 4.332613468170166
dark color, Score: 4.323302745819092
color treat, Sc